**search a user by string embedding**<br>
string => tweet_embedding => user_embedding

In [16]:
import pandas as pd
import numpy as np
# import sys
# sys.path.append("../../scripts/")
from importlib import reload
import TweetToVect2 as TW
import UserToVect as UVect
reload(TW)

<module 'TweetToVect2' from '../../scripts\\TweetToVect2.py'>

In [6]:
users_df = pd.read_pickle('../../datasets/users_vectores.pkl')

In [64]:
users_df.columns

Index(['user_id', 'user_name', 'name', 'description', 'url', 'created_at',
       'geo_enabled', 'verified', 'profile_image_url',
       'profile_image_url_large', 'followers_count', 'friends_count',
       'listed_count', 'favourites_count', 'statuses_count', 'media_count',
       'pinned_tweet', 'tweets_scraped', 'tweets_scraped_at', 'scrapped_at',
       'ignore', 'vector_user', 'n_tweets'],
      dtype='object')

In [17]:
tweet_model = TW.TweetToVect()
tweet_model.load_model(model_path='../../models/words_fast_text.bin')
# user_model = UVect.UserToVect()
# user_model.load_encoder(model_path='../../models/user_to_vect_encoder_b')

In [8]:
pd.options.display.max_colwidth = 200

In [19]:
tweet_model.get_similar_word(positive=['🌈'], topn=5)

[('🏳', 0.9444333910942078),
 ('mesdelorgullo', 0.7774953842163086),
 ('10añosdematrimonioigualitario', 0.7723351716995239),
 ('loveislove', 0.7712818384170532),
 ('diadelorgullolgtbi', 0.7707332372665405)]

In [71]:
users_to_compare = users_df[users_df.n_tweets > 30]

In [72]:
search_terms = ['💚', '🌈', 'peronista', 'ucr', 'periodista', 'conicet', '#juntosporelcambio']
for search_term in search_terms:
    generated_n = 5
    n_words = 5
    similar_words = tweet_model.get_similar_word(positive=[search_term], topn = (generated_n*n_words)-1)
    similar_words = [i[0] for i in similar_words]
    similar_words += [search_term]
    tweets = []
    for i in range(generated_n):
        tweets.append(' '.join(similar_words[i*n_words:(i*n_words)+n_words]))
    search_emb = tweet_model.get_vectors(tweets)
    search_user = user_model.get_vector(search_emb)
    similarities = user_model.pairwise_similarity_vectors([search_user], users_to_compare.vector_user.values.tolist())
    sorted_keys = np.argsort(similarities[0])[::-1]
    sorted_similarity = np.sort(similarities[0])[::-1]
    results = users_to_compare.iloc[sorted_keys, :].head(10)
    results.index = sorted_similarity[:10]
    print('Results for', search_term)
    print('Tags generados:',  tweets)
    display(results[['user_name', 'followers_count','name','description']])

Results for 💚
Tags generados: ['💜 abortolegal2020 abortolegalahora rcmxabortolegal 13jabortolegal', 'abortolegalsl abortolegalessalud abortolegalzn abortolegal8a luchoxabortolegal', 'campabortolegal 13jlamareacrece seraley abortolegalsegurogratuito campabortolegsf', 'quesealey gritoglobalporabortolegal lamareacrece catedrasporabortolegal hastaquesealey', 'queelabortosealey porabortolegal evoluciónesabortolegal congresoverde 💚']


,user_name,followers_count,name,description
0.836513,CampAbortoLegal,125799,#AbortoLegal2020 💚,"Campaña Nacional por el Derecho al Aborto Legal, Seguro y Gratuito. Coordinadora @lac_28"
0.833737,abortolegalmza,4555,Camp. Aborto Legal Mendoza,"Campaña Nacional por el Derecho al Aborto Legal, Seguro y Gratuito Mendoza.\n\nFb: https://t.co/rCMKy48ZPv"
0.828123,PorAbortoLegal,56220,#AʙᴏʀᴛᴏLᴇɢᴀʟ2020 💚,"ESI PARA DECIDIR, ANTICONCEPTIVOS PARA NO ABORTAR #ABORTOLEGAL PARA NO MORIR https://t.co/rbg7C9pRp4\nhttps://t.co/M9cXtx6N8L"
0.822416,CampaAbortoRos,8849,#Abortolegal Rosario 💚,Campaña Nac por el Derecho al Aborto Legal Seguro y Gratuito Rosario Fb: https://t.co/jRejs24TqI…\nInsta: https://t.co/ZiF7aOFazs
0.769440,RedSaludDecidir,16522,Red Profesionales de la salud x Derecho a Decidir,Somos una red de profesionales y de equipos de salud en todo el país. Acompañamos a quienes deciden abortar. En la @CampAbortoLegal. Nuestro recursero en la web
0.727290,CampAbortoCABA,4567,Campaña Nacional por el Derecho al Aborto- CABA💚,"Campaña Nacional por el Derecho al Aborto Legal, Seguro y Gratuito en Ciudad de Buenos Aires"
0.715301,RedAborto,368,Red Aborto Puerto Madryn,"Colectiva feminista que acompaña e informa a personas que desean abortar. Decidir sobre nuestro cuerpo es un derecho, para que se cumpla, nos organizamos."
0.715228,FrenteChaco,660,Frente de Comunicadoras del Chaco,El Frente de Trabajadoras de la Comunicación busca construir una comunicación libre de sexismos y con perspectiva de género
0.708347,celestemacdou,9317,Celeste Mac Dougall,Docente de Historia. Especialista en #ESI. Feminista. \nLuchando por el #AbortoLegal2020 en la @CampAbortoLegal \nActivista en @MujeresMundanas. ¡Basta de ASI!
0.701750,campaniabortoLP,4849,#AbortoLegal2020 #LaPlata 💚🏳️‍🌈,CampañaxAbortoLegal en La Plata\n#SeguimosEnCampaña #AbortoLegalYa\nEDUCACIÓN SEXUAL PARA DECIDIR ANTICONCEPTIVOS PARA NO ABORTAR ABORTO LEGAL PARA NO MORIR


Results for 🌈
Tags generados: ['🏳 mesdelorgullo 10añosdematrimonioigualitario loveislove diadelorgullolgtbi', '10añosmatrimonioigualitario diadelorgullolgtbiq orgullo2020 díadelorgullolgtbi 10anosmatrimonioigualitario', 'diadelorgullolgbt lgbtiqa 10anosdematrimonioigualitario pride2020 matrimonioigualitario', 'diadelorgullo lgbtiqa+ orgullolgtbi orgullolgbt matrimonioigualitariobc', 'pridemonth lgtbifobia orgullo2018 orgullolgtbiq 🌈']


,user_name,followers_count,name,description
0.696488,DiversidadCCARI,1045,CC ARI-Diversidades,Somos el espacio de DDHH y Diversidades de la @ccari_ok.
0.621012,epaulonlgbt,13288,Esteban Paulon 🌈💚,"Socialista. Director Ejecutivo de @politicaslgbt y Presidente del Comité Ejecutivo @redgaylatino Un tipo tranqui, tratando de no complicarla"
0.610098,100porcientoDyD,4116,Cienporcientodiversidad,Organización LGBTI de Argentina. Por una sociedad que nos reconozca con igualdad en nuestras diferencias.\nSomos parte de @OrgulloyluchaAr e @ILGALAC
0.601128,FALGBT,9815,Federación Arg LGBT,"Organización nacional en defensa y promoción de derechos de lesbianas, gays, bisexuales, trans +"
0.558644,PresentesLGBT,24054,Agencia Presentes,"Periodismo, géneros y derechos humanos. Coberturas desde Argentina, Paraguay, Perú, Chile, Honduras, Guatemala, El Salvador, Uruguay y México."
0.554383,GermanBecker_,10708,ɢᴇʀᴍᴀɴ ʙᴇᴄᴋᴇʀ #QuedateEnCasa 🚩,"XXVI. Insomnio, fotografía, redes y algunas boludeces más... sincero, anti moralina e intenso 100pre - 🇦🇷 🏳️‍🌈"
0.515167,generosne,1647,Géneros y Disidencias Nuevo Encuentro - Nacional,Espacio Nacional de Géneros y Disidencias de @NuevoEncuentro_ \nFb: https://t.co/SLm5bl1Gxm
0.509721,almaacrata,6865,amanda alma,🎙Periodista 🏳️‍🌈Lesbik ✂️Feminista 💪🏽Peronista ✌🏽@_Manifiesta_ 🧠 Territorio Liberado para FutuRock 📻en Spotify https://t.co/l3LetyBa9Q
0.482102,gustavopecoraro,14955,Gustavo Pecoraro🇪🇸🏳️‍🌈🇦🇷 #LaPecoTormentosa💚,"Escritor y periodista. Integro la red de podcasts de El Vahído 🎧 Vivo con VIH y soy feliz. Activista LGTBI jubilado, pero con changas. 🇪🇸🏳️‍🌈🇦🇷"
0.479644,AmorEsAmorTw,188,Amor es Amor,Colective Orgullo de Ser // Cultura - Arte - Educación \nVisibilidad \nTrelew - Chubut LGBTIQ+\nhttps://t.co/rdZPDj2vAr


Results for peronista
Tags generados: ['peronismo justicialismo _justicialista peronistan antiperonista', 'justicialista peronistas ebolaperonista peronistatuis peronio', 'pj peronchismo peron kirchnerista peronchas', 'peroncho militancio pibeperonista militancia 501r17deoctubre', 'de_peronista militante antikirchnerista marxisista peronista']


...

,user_name,followers_count,name,description
0.665211,FOPEA,31755,FOPEA,"Foro de Periodismo Argentino. Mejor calidad del periodismo a través de la capacitación, elevación de los estándares éticos y defensa de la libertad de expresión"
0.621790,Adepargentina,9928,ADEPA,Cuenta oficial de la Asociación de Entidades Periodísticas Argentinas. Agrupamos a los medios periodísticos de toda la Argentina.
0.616337,alejandroalfie,14067,Alejandro Alfie,"Lic Cs Comunicación UBA. Mag. Periodismo UdeSA. Profesor adjunto UBA. Tres libros. Periodista Clarín, conductor Contacto Digital y columnista Ventana Abierta."
0.594128,Canal7SiPreBA,8414,Trabajadores de Prensa Canal 7,Comisión Interna y Asamblea de trabajadorxs de prensa de la TV Pública. Nucleados en @SiPreBA
0.590899,periodismoIPR,1042,Periodismo Rosario,Instituto de Periodismo Rosario. Técnico Superior en Periodismo y Técnico Superior en Periodismo Deportivo. 📲 Contacto: 3412556200.
0.583838,PrensaFATPREN,2671,Prensa FATPREN,NaN
0.575936,TrabajadorxsC5N,5715,COMISIÓN INTERNA DE TRABAJADORXS DE C5N 📺,La vía de expresión colectiva OFICIAL de la Comisión Interna de las trabajadoras y trabajadores de #C5N\n#TrabajadorxsC5N\n#CuentaParaSueldosC5N
0.570056,sipreba,24190,Sindicato Prensa BA,-- SiPreBA -- Cuenta oficial del Sindicato de Prensa de Buenos Aires. IG: https://t.co/XLXVBb3vHe FB: https://t.co/b3vlyGGfmY…
0.566091,AguLecchi,3039,Agu Lecchi,Somos los mismos de siempre \n#Temperley #SiPreBA
0.561707,NotiDial,1465,NOTIDIAL,El portal de noticias de radios de argentina. \nprensa@notidial.com / comercial@notidial.com


Results for conicet
Tags generados: ['conicetdialoga científicos ciecsconicet investigadores científic', 'ibr_conicet investigador investigadorxs másconicet investigadoresunsam', 'investigadoras conicetdoc ceil_conicet konicet iibunsam', 'científicxs científicas geac_conicet diadelinvestigadorcientifico investigadora', 'cientificxs conicetlaplata postdoctoral kornblihtt conicet']


,user_name,followers_count,name,description
0.769746,CONICETDialoga,86745,CONICET Dialoga,Twitter Oficial del Consejo Nacional de Investigaciones Científicas y Técnicas.
0.742230,becariosemp,2957,Becarixs Empoderadxs,Trabajadorxs de la ciencia para un desarrollo soberano
0.718256,PortoEdu,2032,Eduardo Porto,Periodista de Ciencia y Tecnología en @RadioCaput y @RadioDelPlata. Editor en @periferiacts. Comunicación y divulgación científica.
0.681179,unsamoficial,25889,UNSAM,"Cuenta oficial de la Universidad Nacional de San Martín (UNSAM), Buenos Aires, Argentina. Buscanos en #UNSAM #ComunidadUNSAM, #OrgulloUNSAM y #EstudiáEnUNSAM"
0.648260,periferiacts,1484,"Periferia Ciencia, Tecnología y Sociedad","Portal de ciencia, tecnología, cultura, sociedad e innovación.\nEditor @PortoEdu"
0.637543,GamarnikLab,8295,Andrea Gamarnik,We are interested in understanding the molecular biology of dengue virus
0.623727,RoloGonzalezOK,1211,Rolando Gonzalez-Jose,"Investigador del CONICET,científico repatriado. Director del Centro Nacional Patagónico-CONICET. Cuenta personal que no refleja posiciones institucionales."
0.614116,pablooesteban,5495,Pablo Esteban,Periodista en @pagina12 y en la @LigadelaCiencia. Divulgación y docencia en @UNQoficial. Ciencia conurbana.
0.603106,ferpeirano,11348,Fernando Peirano 💚,"Presidente Agencia I+D+i (Agencia Nacional de Promoción de la Investigación, el Desarrollo Tecnológico y la Innovación).\n#GrupoCallao #AgendaArgentina"
0.600711,CarlosGrecoOK,1185,Carlos Greco Rector UNSAM,"Cuenta oficial de Carlos Greco, rector de la Universidad Nacional de San Martín (#UNSAM). #SomosUNSAM #ComunidadUNSAM"


Results for #juntosporelcambio
Tags generados: ['juntosporelcambio juntosporelcambioboletacompleta radicalesporelcambio defensoresdelcambio slcambiemos', 'afianzandoelcambio juntossomosimparables juntossomoselcambio cambiemoshoy equipocambiemos', 'ucrencambiemos juntospodemoscambiar llegóelcambio juntosxelcambio defdelcambio', 'cambiemos cambiemoslanus radicalesencambiemos somoselcambio juntosesposible', 'vamosjuntos macripichetto2019listacompleta sisepuede yonoaflojo #juntosporelcambio']


,user_name,followers_count,name,description
0.699972,PROCapitalBA,35914,PRO Capital,Canal Oficial de Twitter de PRO Capital #VamosJuntos
0.696454,canomig,860,Miguel Cano,Anestesiólogo.Asesor en Políticas de Salud y Adm. Pública. Coord.Equipo técnico de Salud. Autoridad del Pro Córdoba. Consejo Administr. Fundación Hacer Córdoba.
0.681774,SiSePuedeARG,8771,SíSePuede,NaN
0.658117,dferrernov,266,Dolores Ferrer Novotny,Cordobesa en Buenos Aires - Periodista y Comunicadora Social - CEO & Founder de CHEQUIA Comunicación - Redes Sociales en @infobae
0.639732,BarbieriVero,11141,Verónica Barbieri,Lic. en Gobierno y RRII / Diputada Provincial / Presidenta JPro Buenos Aires / #JuntosPorElCambio
0.623038,germuller39,622,Muller German,Miembro de la Seccional 4° PRO Cba. Relator de Comisión en el Concejo Deliberante de la Ciudad de Córdoba. Profesional en RRHH y Relaciones Públicas.
0.617393,EzequielPazos,3469,Ezequiel Pazos,Papá de Juana. Concejal de José C. Paz. Abogado. Hincha de River.
0.608343,juntoscambioar,85768,JxC Juntos por el Cambio,Cuenta oficial de Twitter de Juntos por el Cambio. Escribinos por WhatsApp +54 9 11 3000-8602
0.604472,vamosjuntos,5117,Vamos Juntos,"Vamos Juntos es un equipo de 3 millones de vecinos, comprometidos con el crecimiento del cambio y la transformación de la Ciudad de Buenos Aires."
0.604066,carmenpolledo,16721,Carmen Polledo,"DIPUTADA NACIONAL, PRO.(INTERBLOQUE JUNTOS POR EL CAMBIO)"
